In [ ]:
# !pip install xgboost

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np

# 1. Load 20 Newsgroups (subset of categories for speed, optional)
categories = ['alt.atheism', 'comp.graphics', 'sci.med', 'talk.politics.misc']
data = fetch_20newsgroups(subset='all', categories=categories, remove=('headers','footers','quotes'))

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42, stratify=data.target
)

# 2. TF-IDF vectorization
vect = TfidfVectorizer(max_features=20_000, ngram_range=(1,2), stop_words='english')
X_train_tfidf = vect.fit_transform(X_train)
X_test_tfidf  = vect.transform(X_test)

# 3. Fit XGBoost (multi-class)
model = XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
)
model.fit(X_train_tfidf, y_train)

# 4a. Classification report (Precision, Recall, F1 per class)
y_pred = model.predict(X_test_tfidf)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=data.target_names))

# 4b. Macro / Micro F1
macro_f1 = f1_score(y_test, y_pred, average='macro')
micro_f1 = f1_score(y_test, y_pred, average='micro')
print(f"Macro-F1: {macro_f1:.3f}, Micro-F1: {micro_f1:.3f}")

# 4c. AUC-ROC (One-vs-Rest)
y_score = model.predict_proba(X_test_tfidf)
y_test_binarized = label_binarize(y_test, classes=np.arange(len(categories)))
roc_auc_macro = roc_auc_score(y_test_binarized, y_score, average='macro', multi_class='ovr')
roc_auc_micro = roc_auc_score(y_test_binarized, y_score, average='micro', multi_class='ovr')
print(f"ROC-AUC (macro): {roc_auc_macro:.3f}, ROC-AUC (micro): {roc_auc_micro:.3f}")


/Users/hehvince/Desktop/LTU/Cycle 4/D7047E Advanced Deep Learning/.conda/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:04:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Classification Report:
                     precision    recall  f1-score   support

       alt.atheism       0.86      0.69      0.77       200
     comp.graphics       0.93      0.83      0.88       243
           sci.med       0.67      0.90      0.77       248
talk.politics.misc       0.83      0.76      0.80       194

          accuracy                           0.80       885
         macro avg       0.83      0.79      0.80       885
      weighted avg       0.82      0.80      0.80       885

Macro-F1: 0.802, Micro-F1: 0.802
ROC-AUC (macro): 0.960, ROC-AUC (micro): 0.959
